<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Time Series
</h2>
</div>    
</center>

<a id = 'toc'></a>
# Table of Contents

- ### [Calculus](#calculus)
- ### [Exploratory Data Analysis](#eda)
- ### [Model](#model)

<a id = 'calculus'></a>
<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Calculus
</h2>
</div>    
</center>

<div style="display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;">AR (AutoRegressive)
</h2>
</div>

To make a forecast based on our time series we will use the AutoRegressive model. Based on the past values in the series - called lags, AR relies on the time series to being stationary. \
An AR model, that depends on a lag in the past - called an **AR model of order one** or AR(1).

$$Y_{t} = \omega + \phi_{1}Y_{t-1} + \varepsilon_{t}$$

- $Y_{t}$ - Target, $Y_{t-1}$ - Lagged Target, $\varepsilon_{t}$ - Error
- $\omega$ - intercept, $\phi$ - constant(coefficient)

This recursion (lagged target going back to the previous data point) in time goes back until the beginning of the series, so these are called **Long Memory Models**.

$$\begin{aligned}
Y_{t} &= \omega + \phi Y_{t-1} + \varepsilon_{t} \\
Y_{t-1} &= \omega + \phi Y_{t-2} + \varepsilon_{t-1} \\
\downarrow \\
Y_{t} &= \omega^{*} + \phi^{2} Y_{t-2} + \phi \varepsilon_{t-1} + \varepsilon_{t}  \\
\\
Y_{t-2} &= \omega + \phi Y_{t-3} + \varepsilon_{t-2} \\
\downarrow \\
Y_{t} &= \omega^{*} + \phi^{3} Y_{t-3} + \phi^{2} \varepsilon_{t-2} + \phi \varepsilon_{t-1} + \varepsilon_{t} \\
\end{aligned}
$$

If we continued to insert the previous observations, we would arrive at the first observation, and with each step the effect of the previous values would decrease IF $|\phi| < 1$. \
This goes back to the idea of stationary - the dependence of previous observations declines over time.

<div style="display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;">MA (Moving Average)
</h2>
</div>

Instead of looking at the previous lagged target value, we'll make use of the lagged error.\
An MA model, that depends on a lag error in the past - called an **MA model of order one** or MA(1).

$$Y_{t} = \omega + \theta_{1}\varepsilon_{t - 1} + \varepsilon_{t}$$

To avoid the first prediction leak (we don't have a prediction for the first observation), we use the average of the time series.\
MA models are **Short Memory Models** - these errors don't last long into the future.

$$\begin{aligned}
Y_{t - 1} &= \omega + \theta \varepsilon_{t-2} + \varepsilon_{t-1}\\
Y_{t} &= \omega + \theta \varepsilon_{t - 1} + \varepsilon_{t}\\
Y_{t + 1} &= \omega + \theta \varepsilon_{t} + \varepsilon_{t+1}\\
\end{aligned}
$$

This goes back to the idea of stationarity - the dependence of previous observations decline over time.
In **MA(1)** models, the effect of shocks has NO effect on the present if they happened long enough ago.

<div style="display:fill;
           border: 0px;
           border-bottom: 2px solid #AAA;
           font-size:80%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;">ARIMA
</h2>
</div>

**ARIMA** - AR + I + MA

$$Y_{t} = \omega + \phi_{1}Y_{t - 1} + ... + \phi_{p}Y_{t - p} + \theta_{t}\varepsilon_{t-1} + ... + \theta_{q}\varepsilon_{t-q} + \varepsilon_{t}$$

- **I** - integrated. Essentially, you need to make your data stationary (by differencing the time series). Distribution depends only on difference in time, not location in time.

**Example with selected orders:**
ARIMA (1, 1, 1) (p, d, q)
- *d* → $Y_{t} - Y_{t - 1} = W_{t}$
- *p, q* → $W_{t} = \omega + \phi_{1}W_{t-1} + \theta_{1}\varepsilon_{t - 1} + \varepsilon_{t}$

**Deciding how many orders (p, q)**\
Couple of different **techniques to selecting models**:
1. Plotting patterns in correlation
2. Automatic selection techniques

[UP](#toc)

<a id = 'eda'></a>
<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Exploratory Data Analysis
</h2>
</div>    
</center>

[UP](#toc)

<a id = 'model'></a>
<center>
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#212121;
           font-size:110%;
           letter-spacing:0.5px">
<h2 style="padding: 10px;
           color:#FFFFFF;">Model
</h2>
</div>    
</center>

[UP](#toc)